In [ ]:
import cv2
import time
from datetime import datetime
import os
import smtplib
from email.message import EmailMessage

# --- Email Configuration ---
# You MUST replace these with your actual email details.
# DO NOT share your password. If using Gmail, use an app password.
EMAIL_ADDRESS = 's.a.mehta135@gmail.com' 
EMAIL_PASSWORD = 'mdcd vgki octj ujiw' # <<< REPLACE THIS
RECIPIENT_EMAIL = 's.a.mehta135@gmail.com'

def send_notification():
    """Sends an email notification."""
    try:
        msg = EmailMessage()
        msg['Subject'] = '⚠️ Home Surveillance Alert: Activity Detected!'
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = RECIPIENT_EMAIL
        msg.set_content(f"Human activity was detected at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}. Check the video file.")
        
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            smtp.send_message(msg)
            print("Notification email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

# --- Video Surveillance Code ---
face_casacde = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
video = cv2.VideoCapture(0)

# Video recording parameters
out = None
recording = False
notification_sent = False 
last_face_time = None
cooldown_period = 3 

if not os.path.exists("haarcascade_frontalface_default.xml"):
    print("Error: haarcascade_frontalface_default.xml not found. Please download it.")
    exit()

if not video.isOpened():
    print("Error: Could not open video stream.")
    exit()

while True:
    check, frame = video.read()
    if frame is None:
        print("Error: Could not read frame from video stream.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_casacde.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10)

    # Check for faces
    if len(faces) > 0:
        last_face_time = time.time()
        
        if not recording:
            exact_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
            filename = f"activity_detected_{exact_time}.mp4"
            
            frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            
            out = cv2.VideoWriter(filename, fourcc, 20.0, (frame_width, frame_height))
            recording = True
            notification_sent = False 
            print(f"Recording started: {filename}")
        
        if not notification_sent:
            send_notification()
            notification_sent = True
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
    elif recording and (time.time() - last_face_time > cooldown_period):
        out.release()
        recording = False
        print("Recording stopped.")

    if recording:
        out.write(frame)

    cv2.imshow("Home Surveillance", frame)
    key = cv2.waitKey(1)

    if key == ord('q'):
        break

if recording:
    out.release()
video.release()
cv2.destroyAllWindows()